# Developing a 2D GAN to recognise and generate hand-written digits

In this notebook I will be creating a 2D GAN which will be used to create hand-written digits from 0-9. I will be using the M   

In [15]:

import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image

In [16]:
training_steps = 50000    
samples = 200
epochs = 1000 #number of epochs
n = 100 #number of batches to be created
numb_input_layer = 5
training_steps = 50000    


In [17]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))])

Import the MNIST dataset and create our training dataset

In [18]:
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)

Import the MIST dataset and create our test dataset

In [19]:
mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)


Below we can check the length of each of our training and test datasets.

In [20]:
len(mnist_trainset)

60000

In [21]:
len(mnist_testset)

10000

Now we can create our labels

In [22]:
labels_map = {
    0: "0 - Zero",
    1: "1 - One",
    2: "2 - Second",
    3: "3 - Three",
    4: "4 - Four",
    5: "5 - Five",
    6: "6 - Six",
    7: "7 - Seven",
    8: "8 - Eight",
    9: "9 - Nine",
}

In [24]:
'''
figure = plt.figure(figsize=(8, 8))
cols, rows = 3, 3
for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(mnist_trainset), size=(1,)).item()
    img, label = mnist_trainset[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.title(labels_map[label])
    plt.axis("off")
    plt.imshow(img.squeeze(), cmap="gray")
plt.show()
'''

'\nfigure = plt.figure(figsize=(8, 8))\ncols, rows = 3, 3\nfor i in range(1, cols * rows + 1):\n    sample_idx = torch.randint(len(mnist_trainset), size=(1,)).item()\n    img, label = mnist_trainset[sample_idx]\n    figure.add_subplot(rows, cols, i)\n    plt.title(labels_map[label])\n    plt.axis("off")\n    plt.imshow(img.squeeze(), cmap="gray")\nplt.show()\n'

In [25]:
train_loader = torch.utils.data.DataLoader(dataset=mnist_trainset, batch_size=n, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=mnist_testset, batch_size=n, shuffle=False)

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        #hidden layer has 25 neurons
        self.fc1 = nn.Linear(2, 25)
        #output layer only has 1 neuron as it is either 1 or 0
        self.fc2 = nn.Linear(25,25)
        self.fc3 = nn.Linear(25,10)

    def forward(self,x):
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.fc2(x)
        x = torch.relu(x)
        x = self.fc3(x)
        x = torch.sigmoid(x)
        return x